In [ ]:
import pandas as pd
df = pd.read_csv('menuxfootfall.csv')
print(df.head())

In [ ]:
import pandas as pd

# Concatenate values from the four columns into a new column 'merged_column'
df['menu'] = df.apply(lambda row: ''.join(str(row[col]) for col in ['breakfast_merge', 'lunch_merge', 'snacks_merge', 'dinner_merge']), axis=1)

# Display the new 'merged_column'

# Function to clean and split menu items
def clean_and_split_menu(menu_string):
    # Replace 'nan' with an empty string and strip extra spaces
    clean_string = menu_string.replace('nan', '').strip()
    # Split the menu items and remove extra spaces
    menu_items = [item.strip() for item in clean_string.split(',') if item.strip()]
    return menu_items

# Clean and split the 'menu' column
df['menu'] = df['menu'].astype(str)  # Ensure all entries are strings
menu_lists = df['menu'].apply(clean_and_split_menu)

# Create a list of all unique menu items
all_menu_items = set(item for sublist in menu_lists for item in sublist)

# Initialize a dictionary to hold our one-hot encoded data
one_hot_encoded_data = {item: [] for item in all_menu_items}

# Populate the dictionary with one-hot encoded values for each row
for menu_items in menu_lists:
    for item in all_menu_items:
        one_hot_encoded_data[item].append(int(item in menu_items))

# Convert the dictionary to a DataFrame
one_hot_encoded_df = pd.DataFrame(one_hot_encoded_data)

# Combine the original DataFrame with the new one-hot encoded DataFrame
final_df = pd.concat([df, one_hot_encoded_df], axis=1)

# Now `final_df` contains the original data along with one-hot encoded menu items
print(final_df['menu'].head())
final_df.to_csv('columns.csv')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('columns.csv')

# Prepare the data for the model, dropping all non-numeric and target columns
drop_columns = ['Date', 'breakfast_merge', 'snacks_merge', 'lunch_merge', 'dinner_merge', 'menu', 'footfall']
X = df.drop(columns=drop_columns)
y = df['footfall']

# Handling categorical variables
categorical_features = ['day', 'month', 'meal_type']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Define which columns are numeric for the ColumnTransformer to work on
numeric_features = X.columns.difference(categorical_features).tolist()

# Combine categorical and numerical transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', StandardScaler(), numeric_features)
    ]
)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply preprocessing, which now only scales numeric features and one-hot encodes categorical features
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# The input shape is the number of features after transformation
input_shape = X_train.shape[1]

# Building the neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(input_shape,)),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with validation split
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Plotting the training and validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss During Training')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Save the final model
#model.save('final_model.h5')


In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the dataset
df = pd.read_csv('menuxfootfall.csv')


# This step is crucial as PCA is sensitive to the variances of the initial variables
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df.select_dtypes(include=[np.number]))

# Applying PCA to retain 95% of the variance
pca = PCA(n_components=0.95)
pca_data = pca.fit_transform(scaled_data)

# Convert the PCA results into a DataFrame
pca_df = pd.DataFrame(data=pca_data)


# Save the PCA-reduced data to a new CSV file if needed
pca_df.to_csv('pcad.csv', index=False)

print(pca_df.head())


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('menuxfootfall.csv')

# Drop non-numeric columns and the target variable
drop_columns = ['Date', 'breakfast_merge', 'snacks_merge', 'lunch_merge', 'dinner_merge', 'menu']
X = df.drop(columns=drop_columns)
y = df['footfall']

# Standardize the data before applying PCA
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)

# Applying PCA
pca = PCA(n_components=0.95)  # Keep 95% of variance
X_pca = pca.fit_transform(scaled_data)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Building the neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with validation split
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Plotting the training and validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss During Training')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('menuxfootfall.csv')

# Drop non-numeric columns and the target variable
drop_columns = ['Date', 'breakfast_merge', 'snacks_merge', 'lunch_merge', 'dinner_merge']
X = df.drop(columns=drop_columns)
y = df['footfall']

# Standardize the data before applying PCA
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)

# Applying PCA
pca = PCA(n_components=0.95)  # Keep 95% of variance
X_pca = pca.fit_transform(scaled_data)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Building the neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with validation split
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Plotting the training and validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss During Training')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()


